<a href="https://colab.research.google.com/github/dmassonfr/Spotify_hit_predictor/blob/prep_presentation/Baseline_model_on_Final_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline model on Final dataset

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
from google.colab import files
uploaded = files.upload()

Saving final_dataset_3.csv to final_dataset_3.csv


In [8]:
import io
data = pd.read_csv(io.BytesIO(uploaded['final_dataset_3.csv']))

<ipython-input-8-e7336a064243>:2: DtypeWarning: Columns (22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(io.BytesIO(uploaded['final_dataset_3.csv']))


In [9]:
data.shape

(447490, 24)

In [10]:
data.head(1)

,id,name,popularity,duration_ms,explicit,artists,release_date,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,id_artists,followers,genres,artist_popularity,preview_url,cover_album
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,Uli,1922-02-22,0.645,0.445,0,...,0.744,0.151,0.127,104.851,45tIt06XoI0Iio4LBEVpls,91.0,NaN,4,NaN,NaN


In [11]:
data['release_year'] = [int(i.split('-')[0]) for i in data['release_date']]

In [12]:
data=data.drop(columns= ['id_artists','id','name','artists','preview_url','cover_album','genres','release_date'])


In [13]:
data.head(1)

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,followers,artist_popularity,release_year
0,6,126903,0,0.645,0.445,0,-13.338,1,0.451,0.674,0.744,0.151,0.127,104.851,91.0,4,1922


In [14]:
data['followers'] = data['followers'].fillna(0)

In [15]:
data.isna().sum()

popularity           0
duration_ms          0
explicit             0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
followers            0
artist_popularity    0
release_year         0
dtype: int64

In [16]:
y= data['popularity']

In [17]:
X=data.drop(columns= 'popularity')

In [18]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,followers,artist_popularity,release_year
popularity,1.000000,0.028443,0.228952,0.207536,0.301528,0.015706,0.321434,-0.041307,0.003931,-0.380277,-0.229574,-0.043958,-0.001680,0.066489,0.253697,0.525471,0.609825
duration_ms,0.028443,1.000000,-0.020179,-0.124909,0.021004,0.005855,-0.003354,-0.027479,-0.142847,-0.066095,0.067384,0.001178,-0.163686,0.000484,0.029052,0.001952,0.049151
explicit,0.228952,-0.020179,1.000000,0.160505,0.122125,0.010891,0.137307,-0.052012,0.116800,-0.152811,-0.068983,-0.018132,-0.019193,0.006003,0.099597,0.122860,0.221868
danceability,0.207536,-0.124909,0.160505,1.000000,0.247283,0.017833,0.269712,-0.043330,0.192384,-0.239154,-0.231459,-0.107755,0.527828,-0.042363,-0.008211,0.010429,0.242005
energy,0.301528,0.021004,0.122125,0.247283,1.000000,0.035326,0.759741,-0.064690,0.009155,-0.712233,-0.196415,0.132658,0.367858,0.222510,0.060032,0.107409,0.447560
key,0.015706,0.005855,0.010891,0.017833,0.035326,1.000000,0.024430,-0.130758,0.001159,-0.026634,-0.004835,-0.007693,0.019209,0.004558,0.002626,0.001955,0.021702
loudness,0.321434,-0.003354,0.137307,0.269712,0.759741,0.024430,1.000000,-0.041367,-0.090766,-0.519229,-0.336391,0.034082,0.277231,0.182342,0.071273,0.063155,0.432180
mode,-0.041307,-0.027479,-0.052012,-0.043330,-0.064690,-0.130758,-0.041367,1.000000,-0.021424,0.057916,-0.009700,0.010679,0.012857,0.008582,0.004796,0.000078,-0.085150
speechiness,0.003931,-0.142847,0.116800,0.192384,0.009155,0.001159,-0.090766,-0.021424,1.000000,0.060741,-0.101125,0.233273,0.055092,-0.087862,-0.024331,0.125518,-0.014779
acousticness,-0.380277,-0.066095,-0.152811,-0.239154,-0.712233,-0.026634,-0.519229,0.057916,0.060741,1.000000,0.199927,-0.004200,-0.173641,-0.193563,-0.082008,-0.156331,-0.527949


In [19]:
#Dropping features with less than 5% of correlation with track_popularity
X= X.drop(columns=['duration_ms','key','mode','speechiness','valence'])

In [20]:
X.head(1)

,explicit,danceability,energy,loudness,acousticness,instrumentalness,liveness,tempo,followers,artist_popularity,release_year
0,0,0.645,0.445,-13.338,0.674,0.744,0.151,104.851,91.0,4,1922


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

scaler= MinMaxScaler()

#Fit the model into X_train
X_train_scaled = scaler.fit(X_train)

#Transform X_train data
X_train_scaled = scaler.transform(X_train)

# Change X_train_scaled to a Dataframe
X_train_scaled = pd.DataFrame(X_train_scaled)

#Fit the model into X_test
X_test_scaled = scaler.fit(X_test)

#Transform X_test data
X_test_scaled = scaler.transform(X_test)

# Change X_test_scaled to a Dataframe
X_test_scaled = pd.DataFrame(X_test)

In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_absolute_error

# Hyperparameter Grid
n_neighbors = range(5,31,5)
p = [1,2,3]

# Instantiate model
model = KNeighborsRegressor()

# Instantiate Grid Search
grid = GridSearchCV(estimator = model,
                    param_grid = {'n_neighbors': n_neighbors, 'p': p},
                    scoring='r2',
                    cv=5,
                    n_jobs=-1)

In [23]:
# Fit data to Grid Search
grid.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid={'n_neighbors': range(5, 31, 5), 'p': [1, 2, 3]},
             scoring='r2')

In [24]:
grid.best_params_

{'n_neighbors': 30, 'p': 1}

In [25]:
grid.best_score_

0.6251164562607817